In [ ]:
import urllib.request

def extract_data(url, filename):
    try:
        urllib.request.urlretrieve(url, filename)
    except Exception as e:
        print(e)

In [ ]:
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/c9509ab4-6f6d-4b97-979a-0cf2a10c922b/download/tmphrybkxuh.csv","data/data_2015.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/b7ea6b1b-3ca4-4c5b-9713-6dc1db52379a/download/tmpzxzxeqfb.csv","data/data_2016.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/30022137-709d-465e-baae-ca155b51927d/download/tmpzccn8u4q.csv","data/data_2017.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv","data/data_2018.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv","data/data_2019.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv","data/data_2020.csv")

In [ ]:
arquivos = [
    "data/data_2015.csv",
    "data/data_2016.csv",
    "data/data_2017.csv",
    "data/data_2018.csv",
    "data/data_2019.csv",
    "data/data_2020.csv",
]

In [ ]:
dfs = {}

In [ ]:
import pandas as pd

for arquivo in arquivos:
    ano = arquivo.split("_")[-1].split(".")[0]
    dfs[ano] = pd.read_csv(arquivo)

In [18]:
dfs["2018"].head

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
import boto3
import os
import dotenv

dotenv.load_dotenv()

AWS_API_KEY = os.environ.get("AWS_API_KEY")
AWS_SECRET_KEY = os.environ.get("AWS_SECRET_KEY")
REGION_NAME = os.environ.get("REGION_NAME")

boto3.setup_default_session(
    aws_access_key_id = AWS_API_KEY,
    aws_secret_access_key = AWS_SECRET_KEY,
    region_name = REGION_NAME
)

s3 = boto3.client("s3")

In [ ]:
content = """
Olá, S3
"""

with open("hello-s3.txt","w+") as f:
    f.write(content)


In [ ]:
s3.upload_file("hello-s3.txt","databoston-aws","bronze/hello-s3.txt")

In [22]:
from io import BytesIO

for ano, df in dfs.items():
    parquet_buffer = BytesIO()
    df.to_parquet(parquet_buffer)

    s3.put_object(
        Bucket="databoston-aws",
        Key=f"bronze/data_{ano}.parquet",
        Body=parquet_buffer.getvalue(),
    )

In [ ]:
response = s3.list_objects(Bucket="databoston-aws")

In [ ]:
keys = [obj["Key"] for obj in response ["Contents"]]
print(keys)